<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Máquinas de Aprendizaje II-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Máquinas de Aprendizaje </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Introducción a librerías comunes de *Machine Learning*:
    * Pandas
    * Numpy
    * Sklearn
    * Matplotlib,..
* Clasificación y regresión sobre texto.
* Implementación a mano de regresión lineal.
* Algoritmo Gradiente descendente
 

** Formalidades **  
* Equipos de trabajo de: 2 personas
* Se debe preparar un (breve) Jupyter/IPython notebook que explique la actividad realizada y las conclusiones del trabajo
* Fecha de entrega y discusión: 5 de Octubre.
* Formato de entrega: envı́o de link Github al correo electrónico del ayudante (*<francisco.mena.13@sansano.usm.cl>*) , incluyendo al profesor en copia (*<jnancu@inf.utfsm.cl>*). Por favor especificar el siguiente asunto: [Tarea0-INF393-II-2018]

<hr style="height:2px;border:none"/>

La tarea se divide en secciones:

[1.](#primero) Sentiment Analysis  
[2.](#segundo) Job Salary Prediction  
[3.](#tercero) Linear Regression by hand

<a id="primero"></a>
## 1. Sentiment Analysis [Sin evaluación]
El análisis de emociones o sentimientos se refiere al proceso de extraer información acerca de la actitud
que una persona (o grupo de ellas) manifiesta, en un determinado medio o formato digital, con respecto a un
tópico o contexto de comunicación. Uno de los casos más estudiados corresponde a determinar la polaridad
de un trozo de texto, es decir, clasificar una determinada evaluación escrita (*review*), en que una persona
manifiesta una opinión, como *positiva*, *negativa* o *neutral*. Esto también ha sido extendido a otros medios, como lo es analizar la polaridad de textos en redes sociales.  

<img src="https://cdn.urgente24.com/sites/default/files/notas/2018/01/18/twitter.png" title="Title text" width="20%" height="20%" />


La conocida red social Twitter tiene una gran cantidad de usuarios, por lo que la información se genera a
cada segundo, donde el análisis de texto se ha aplicado fuertemente a estos medios sociales. La dificultad de
este problema radica en el carácter altamente ambiguo e informal del lenguaje que utilizan naturalmente las
personas ası́ como el manejo de negaciones, sarcasmo y abreviaciones en una frase.


Los datos pueden ser descargados ejecutando el siguiente código en sistema Unix:
```
wget -O train_data.csv http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train
wget -O test_data.csv http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev
```

Para construir un clasificador que determine automáticamente la polaridad de un trozo de texto, vamos a necesitar representar los textos $\{d_i\}_{i=1}^n$ disponibles como vectores de caracterı́sticas (features). Para aumentar la eficacia de las caracterı́sticas extraı́das es conveniente ejecutar algunas técnicas de pre-procesamiento básicas como: pasar todo el texto a minúsculas (lower-casing), eliminar signos de puntuación
y eliminar palabras sin significado como artı́culos, pronombres y preposiciones (*stop word removal*). Otra
técnica que suele ser útil para obtener buenas caracterı́sticas (*features*) es la lematización, es decir, la
reducción de todas las palabras a su tronco léxico base. Una técnica similar y más utilizada en la práctica es el stemming. Varias de éstas están implementadas en la libreria nltk [13] para python.


> a) Construya un dataframe con los datos a analizar. Determine cuántas clases existen, cuántos registros
por clase y describa el dataset.
``` python
import pandas as pd
ftr = open("train_data.csv", "r")
fts = open("test_data.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
df_train = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_train['Sentiment'] = pd.to_numeric(df_train['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
df_test = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_test['Sentiment'] = pd.to_numeric(df_test['Sentiment'])
df_train.shape
```

> b) Construya un conjunto de validación (también conocido como *hold out validation*), a través de una máscara aleatoria, para verificar los resultados de los algoritmos.
``` python
import numpy as np
msk = np.random.rand(len(df_train)) < 0.8
df_new_train = df_train[msk]
df_val = df_train[~msk]
```

> c) Implemente y explique un pre-procesamiento para los tweets para dejarlos en un formato estándarizado
en el cual se podrán trabajar.
```python
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer
def word_extractor(text):
    wordlemmatizer = WordNetLemmatizer()
    wordstemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordstemmer.stem(word.lower())for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words
word_extractor("I love to eat cake")
word_extractor("I love eating cake")
word_extractor("I loved eating the cake")
word_extractor("I do not love eating cake")
word_extractor("I don't love eating cake")
...
wordlemmatizer.lemmatize(word.lower()) #can use this
```

> d) Utilizando la función *CountVectorizer* de la librerı́a sklearn genere una representación vectorial del texto de entrenamiento y del conjunto que usaremos para realizar la validación. Esta función consiste en contar cuántas veces aparecen ciertos términos/palabras en el texto a través de un vocabulario que construiremos mediante la unión de todas las palabras que observemos en los textos que tenemos a disposición. Explore el vocabulario utilizado y determine cuáles son las palabras más frecuentes en el conjunto de entrenamiento y validación.
``` python
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
texts_train = [word_extractor(text) for text in df_new_train.Text]
texts_val = [word_extractor(text) for text in df_val.Text]
texts_test = [word_extractor(text) for text in df_test.Text]
vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))
features_train = vectorizer.transform(texts_train)
features_val = vectorizer.transform(texts_val)
features_test = vectorizer.transform(texts_test)
labels_train = np.asarray((df_new_train.Sentiment.astype(float)+1)/2.0)
labels_val = np.asarray((df_val.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((df_test.Sentiment.astype(float)+1)/2.0)
vocab = vectorizer.get_feature_names()
dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))
for tag, count in zip(vocab, dist):
    print count, tag
```

> e) Construya una función que entrene/ajuste un modelo de Regresión Logı́stica y mida el error de predicción obtenido sobre los datos de entrenamiento y validación. Experimente con variar la representación o preprocesamiento, por ejemplo el efecto de filtrar *stopwords* y de eliminar este paso de
pre-procesamiento tı́pico. Determine además, qué representación obtiene un mejor resultado: si aquella obtenida vı́a lematización o aquella obtenida vı́a *stemming*. Seleccione el mejor modelo y mida sobre *test*. Comente
``` python
from sklearn.linear_model import LogisticRegression
def do_LOGIT(x,y,xv,yv):
    model = LogisticRegression()
    model = model.fit(x, y)
    print("Accuracy under training: ",model.score(x,y))
    print("Accuracy under validation: ",model.score(xv,yv))
    return model
log_model = do_LOGIT(features_train,labels_train,features_val,labels_val)
```

> f) Finalmente, tome un subconjunto aleatorio de los textos de prueba y analice las predicciones del modelo (explore las predicciones, ası́ como las probabilidades que el clasificador asigna a cada clase)

<a id="segundo"></a>
## 2. Job Salary Prediction

En esta sección se trabajará con el problema de predecir el salario que ofrece un anuncio en internet a través unicamente del texto del anuncio. El dataset es ofrecido por *Adzuna* como una competencia en la plataforma más grande de *data science* Kaggle, a través del siguiente __[link](https://www.kaggle.com/c/job-salary-prediction)__. El objetivo de la competencia, según *Adzuna*, es el de tener un motor que pueda predecir el salario de cualquier anuncio de trabajo en Reunio Unido, para poder mejorar la experiencia de los usuarios que buscan trabajos, ya que así pueden filtrar sin que el mismo empleador señale explícitamente cuánto paga.

<img src="http://s5047.pcdn.co/wp-content/uploads/2013/05/salary-prediction-engine-v2.png" title="Title text" width="50%"/>

La métrica de evaluación de la competencia es MAE (*mean absolute error*):

$$
L = \frac{1}{N} \sum_i^N  \mid y_i - \hat{y}_i \mid
$$


Para descargar los datos a utilizar (*Train_rev1*) debe estar registrado en la plataforma de Kaggle. Se cuenta con cientos de miles registros con textos sin procesar, es decir, no están estructurados.


> a) Carge los datos *csv* de entrenamiento y cree un conjunto de validación con los últimos 10 mil datos en un dataframe de *pandas*. Describa los datos, apóyese de gráficos ¿Cuántos datos hay en cada conjunto?  
```python
import pandas as pd
df = pd.read_csv("Train_rev1.csv")
df_train = df.iloc[:-10000]
df_val = df.iloc[-10000:]
...#load other sets
df.head()
```
*Recuerde que si no puede trabajar con todos los datos, debido a su volumen, puede muestrear*.


> b) Extraiga los datos de cada conjunto con los que trabajará, el *input* $X$, los textos, y el *output* $y$, los salarios.
```python
text = df.FullDescription
salary = df.SalaryNormalized
```

> c) Realice un pre-procesamiento a los datos brutos de texto para extraer características y generar la representación de los datos de entrada al modelo $\vec{x}$. Comente sobre lo realizado.

> d) Intente resolver el problema enfrentándolo como regresión con el modelo de regresión lineal ordinaria en *sklearn*. ¿Qué es lo que hace *fit_intercept=True*? Evalúe la función objetivo (F.O.) utilizada y la métrica de la competencia (*mean absolute error*) en ambos conjuntos generados en el punto a). Comente lo observado.
```python
from sklearn.linear_model import LinearRegression as LR
model = LR(fit_intercept=True, normalize=False)
model.fit(X_train,y_train)
... #measure F.O.
from sklearn.metrics import mean_absolute_error #measure MAE
print("MAE on train: ",mean_absolute_error(y_train, model.predict(X_train)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(X_val)))
```

> e) Intente mejorar sus resultados en base a la métrica de la competencia (MAE) sobre el conjunto de validación. Comente sobre lo realizado.  
*No se le pide que imite los resultados ganadores de la competencia (MAE de 3400), sino que mejore lo ya alcanzado siendo creativo.*
<div class="alert alert-warning"> HINT: Una opción es cambiar el *approach* de resolución desde regresión a casificación o trabajo sobre los datos (tal como limpiarlos).</div>


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv("Train_rev1.csv")
df.head()

El dataset se compone de 12 atributos, los cuales son:

+ Id: identificador único para cada anuncio laboral.
+ Title: título del aviso laboral.
+ FullDescription: texto completo del anuncio laboral.
+ LocationRaw: ubicación del empleo.
+ LocationNormalized: ubicación normalizada.
+ ContractType: tipo de contrato.
+ ContractTime: duración del contrato.
+ Company: nombre del empleador. 
+ Category: categoría del empleo.
+ SalaryRaw: salario.
+ SalaryNormalized: salario anual.
+ SourceName: nombre del sitio o anunciante que provee el aviso laboral.

In [ ]:
df.shape

El dataset contiene 244.768 registros, cada uno con 12 atributos distintos.

In [ ]:
df.info()

Del resumen del dataset se puede observar que ningún atributo cuenta con registros nulos. Además, dos de los atributos son de tipo númerico y el resto de los atributos son textos.

In [ ]:
df.describe()

De los atributos númericos, el que nos interesa para las predicciones es *SalaryNormalized*. El método `describe()` muestra las estadísticas para dicho atributo.

Para estudiar el comportamiento del atributo *SalaryNormalized* se realiza un boxplot.

In [ ]:
sns.boxplot(list(df.SalaryNormalized))
plt.ylabel('Datos')
plt.xlabel('SalaryNormalized')
plt.title('Salario')
plt.show()

Es evidente que los datos presentan un sesgo negativo bastante significativo. Tambien se aprecia la presencia de muchos outliers.

Para un primer acercamiento al problema, el dataset de entrenamiento se separará en un conjunto de entrenamiento con 15.000 registros y un conjunto de validación con 10.000 registros.

In [ ]:
rows = np.random.choice(df.index.values, 15000, replace=False) #Elige 15000 filas al azar del df, no se repiten.

df_train = df.loc[rows]
df_val = df.iloc[-10000:]

print("Largo entrenamiento:", len(df_train.index), "|", "Largo validación:", len(df_val.index))

> b) Extraiga los datos de cada conjunto con los que trabajará, el *input* $X$, los textos, y el *output* $Y$, los salarios.

A continuación, se separan los conjuntos de entrenamiento y validación en los respectivos inputs y outputs. Siendo el input $X$ el atributo *FullDescription* y el output $Y$ el atributo *SalaryNormalized*.

In [ ]:
texto_train = df_train.FullDescription
salario_train = df_train.SalaryNormalized

texto_val = df_val.FullDescription
salario_val = df_val.SalaryNormalized

> c) Realice un pre-procesamiento a los datos brutos de texto para extraer características y generar la representación de los datos de entrada al modelo $\vec{x}$. Comente sobre lo realizado.

Para pre-procesar el texto se decidió utilizar la lematización debido a que considera el contexto de las palabras y puede resultar útil para el proceso de aprendizaje. Tambien se filtran las stopwords o palabras que no aportan significado al texto, mediante expresiones regulares se eliminan patrones como \*\*\*\* entre otros y se convierten las palabras a minúsculas. Por último, debido a la naturaleza del problema, se decide omitir las palabras cuyo largo sea mayor a 20 carácteres.

A continuación, se define la función `word_extractor` encargada del pre-procesamiento.

In [ ]:
palabrasComunes = stopwords.words('english')

def word_extractor(text, commonwords):
    wordlemmatizer = WordNetLemmatizer()
    text = re.sub('\\*\\*\\*\\*|\\.|\\,|\\/|\\|', '',text) #elimina los **** del texto
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower())for word in word_tokenize(text) ]
    
    for word in wordtokens:
        if word not in commonwords and len(word) < 20:
            words+=" "+word
    return words

Se construyen las estructuras que serán utilizadas para el proceso de aprendizaje del modelo usando la función definida anteriormente.

In [ ]:
start = time.time()

texts_train = [word_extractor(text, palabrasComunes) for text in texto_train]
texts_val = [word_extractor(text, palabrasComunes) for text in texto_val]

vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))

features_train = vectorizer.transform(texts_train)
features_val = vectorizer.transform(texts_val)

labels_train = np.asarray(salario_train)
labels_val = np.asarray(salario_val)

print("Tiempo que tarda en ejecutarse:", round(time.time()-start, 1), "segundos")

In [ ]:
features_train

Se tiene una matriz dispersa para el input de entrenamiento.

In [ ]:
vocab = vectorizer.get_feature_names()
dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))
lista = zip(vocab, dist)
lista_ordenada = sorted(lista, key=lambda x: x[1], reverse=True)
lista_ordenada

En la celda anterior se muestran las palabras más frecuentes del vocabulario generado.

> d) Intente resolver el problema enfrentándolo como regresión con el modelo de regresión lineal ordinaria en *sklearn*. ¿Qué es lo que hace *fit_intercept=True*? Evalúe la función objetivo (F.O.) utilizada y la métrica de la competencia (*mean absolute error*) en ambos conjuntos generados en el punto a). Comente lo observado.

Se procede a trabajar con un modelo de regresión lineal ordinaria. Para esto se utiliza el método `LinearRegression()` con parámetros `fit_intercept = True` y `normalize = False`. *fit_intercept* determina si se calcula el intercepto $\beta_0$ de la regresión lineal y solo debiese ser falso si se espera que los datos estén previamente centrados, por otra parte,*normalize* determina si se normalizan los regresores $X$ antes de realizar la regresión lineal.

In [ ]:
start = time.time()

model = LinearRegression(fit_intercept=True, normalize=False)
model.fit(features_train, labels_train)

#measure MAE
print("MAE sobre entrenamiento: ", round(mean_absolute_error(labels_train, model.predict(features_train)), 1))
print("MAE sobre validación: ", round(mean_absolute_error(labels_val, model.predict(features_val)), 1))

print("Tiempo que tarda en ejecutarse:", round(time.time()-start, 1), "segundos")

La métrica utilizada para evaluar el rendimiento del modelo corresponde al Error Absoluto Medio (MAE), cuya fórmula es:

$$
L = \frac{1}{N} \sum_i^N  \mid y_i - \hat{y}_i \mid
$$

Se interpreta como la distancia media entre las predicciones y los valores reales. En este caso, para el conjunto de validación se obtuvo un MAE bastante alto. Este valor puede deberse a usar pocos datos para el entrenamiento y/o un pre-procesamiento que no es adecuado.

En el siguiente inciso se propone una serie de cambios para mejorar el rendimiento del modelo y se discute el resultado obtenido.

> e) Intente mejorar sus resultados en base a la métrica de la competencia (MAE) sobre el conjunto de validación. Comente sobre lo realizado.  
*No se le pide que imite los resultados ganadores de la competencia (MAE de 3400), sino que mejore lo ya alcanzado siendo creativo.*

Los cambios que se proponen consisten en:
+ Eliminar los outliers para el conjunto de entrenamiento (el conjunto de validación se mantiene igual que en los incisos anteriores).
+ Eliminar palabras/símbolos que podrían estar generando ruido.

Primero se carga el dataset nuevo omitiendo los registros cuyo *SalaryNormalized* sea mayor a 68000. Este numero se obtiene sel boxplot realizado en los incisos anteriores.

In [ ]:
df_nuevo = df[df.SalaryNormalized <= 68000]
df_nuevo.head()

A continuación, se construye nuevamente el boxplot

In [ ]:
sns.boxplot(list(df_nuevo.SalaryNormalized))
plt.ylabel('Datos')
plt.xlabel('SalaryNormalized')
plt.title('Salario sin outliers')
plt.show()

Como se puede apreciar, no queda ningun outlier y la distribución de los datos se asemeja más a una normal. Esto facilitará el proceso de aprendizaje del modelo

Luego, se repite el procedimiento realizado en los incisos anteriores usando el nuevo dataset para evaluar si existe una mejora respecto al primer modelo.

In [ ]:
rows_nuevo = np.random.choice(df_nuevo.index.values, 15000, replace=False) #Elige 15000 filas al azar del df_nuevo, no se repiten.

df_train_nuevo = df_nuevo.loc[rows_nuevo] ##el conjunto de entrenamiento se obtiene del dataset sin outliers
df_val_nuevo = df.iloc[-10000:] ##El conjunto de validación es el mismo que el anterior

print("Largo entrenamiento:", len(df_train_nuevo.index), "|", "Largo validación:", len(df_val_nuevo.index))

In [ ]:
texto_train_nuevo = df_train_nuevo.FullDescription
salario_train_nuevo = df_train_nuevo.SalaryNormalized

texto_val_nuevo = df_val_nuevo.FullDescription
salario_val_nuevo = df_val_nuevo.SalaryNormalized

In [ ]:
palabrasComunes = stopwords.words('english')
palabrasRuido = [":", ".", ",", ";", "12", "10", ]

def word_extractor2(text, commonWords, noiseWords):
    wordlemmatizer = WordNetLemmatizer()
    text = re.sub('\\*\\*\\*\\*|\\/|\\|', '',text)
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower())for word in word_tokenize(text) ]
    
    for word in wordtokens:
        if word not in commonWords and word not in noiseWords and len(word) < 20:
            words+=" "+word
    return words

In [ ]:
start = time.time()

texts_train_nuevo = [word_extractor2(text, palabrasComunes, palabrasRuido) for text in texto_train_nuevo]
texts_val_nuevo = [word_extractor2(text, palabrasComunes, palabrasRuido) for text in texto_val_nuevo]

vectorizer2 = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer2.fit(np.asarray(texts_train_nuevo))

features_train_nuevo = vectorizer2.transform(texts_train_nuevo)
features_val_nuevo = vectorizer2.transform(texts_val_nuevo)

labels_train_nuevo = np.asarray(salario_train_nuevo)
labels_val_nuevo = np.asarray(salario_val_nuevo)

print("Tiempo que tarda en ejecutarse:", round(time.time()-start, 1), "segundos")

In [ ]:
start = time.time()

model_nuevo = LinearRegression(fit_intercept=True, normalize=False)
model_nuevo.fit(features_train_nuevo, labels_train_nuevo)

#measure MAE
print("MAE sobre entrenamiento: ", round(mean_absolute_error(labels_train_nuevo, model_nuevo.predict(features_train_nuevo)), 1))
print("MAE sobre validación: ", round(mean_absolute_error(labels_val_nuevo, model_nuevo.predict(features_val_nuevo)), 1))

print("Tiempo que tarda en ejecutarse:", round(time.time()-start, 1), "segundos")

En general, se obtiene una mejora considerable respecto al modelo anterior, llegando a haber una diferencia de 6000 en el MAE. Sin embargo, existen ocasiones en que el primer modelo obtiene mejores resultados que el segundo.

La idea de eliminar los outliers es hacer que el modelo sea mas estable, realizando así mejores predicciones.

<a id="tercero"></a>
## 3. Linear Regression by hand

En esta sección se le pedirá que implemente la regresión lineal ordinaria a través del algoritmo SGD (*Stochastic Gradient Descend*) para encontrar los parámetros de la regresión a través de este algoritmo de manera iterativa. La técnica de SGD es sin duda dominante al momento de entrenar modelos en máquinas de aprendizaje cuando la solución no tiene un óptimo derivable analíticamente, en este caso la regresión lineal ordinaria que trabajaremos si tiene óptimo anaĺitico, sin embargo, se le pedirá que compare este caso con fines pedagógicos.

* Regresión lineal ordinaria:
$$
\hat{y} = f(\vec{x}) =\vec{\beta}^T\cdot \vec{x}
$$

* Función objetivo:
$$
Loss = \frac{1}{N} \sum_i^N ( y_i - \hat{y}_i )^2
$$


* Algoritmo SGD para regresión lineal ordinaria:
$$ \vec{\beta}^{(t+1)} \leftarrow \vec{\beta}^{(t)} - \eta \nabla_{\vec{\beta}^{(t)}} Loss $$

Para lo que sigue de la actividad sólo podrá utilizar *numpy* (para operaciones de algebra lineal).

> a) Escriba una función que calcule la función de pérdida, error cuadrático medio (MSE - *mean squared error*), para un dato o para un conjunto de datos.

> b) Escriba una función que calcule el gradiente (derivada) de la función de pérdida anterior, para un dato o para un conjunto de datos. *Escriba explícitamente la derivada (gradiente)*.

> c) Escriba una función que calcule los parámetros de una regresión lineal simple de manera analítica (es decir el mínimo global). 

<div class="alert alert-warning"> Hint: ésto debería ser la ecuación desglosada de la derivada igual a 0, la cual incluye $(X^TX)^{-1}$</div>


> d) Ahora escriba un programa que permita entrenar una regresión lineal a través del algoritmo SGD mostrado en la ecuación del algoritmo SGD, es decir, que de manera iterativa, vaya tomando un dato a la vez, y actualizando el parámetro $\beta$ a través del gradiente descendiente de la función de pérdida de la regresión lineal ordinaria, de la pregunta b).

> e) Demuestre que sus programas funcionan en un problema de regresión simple. Para esto utilice el dataset **Boston Housing** , disponible a través de la librería __[*sklearn*](http://scikit-learn.org)__, el cual corresponde a el precio de diferentes casas en Boston además de distintas características relevantes respecto al lugar, como por ejemplo el crimen en la ciudad, el número de habitaciones, que tan vieja es, distancia a lugares relevantes, entre otros. Éstas características deben combinarse linealmente para estimar el precio de la casa.
<div class="alert alert-block alert-info">Es una buena práctica el normalizar los datos antes de trabajar con el modelo</div>

```python
from sklearn.datasets import load_boston
X_train,y_train = load_boston(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
```
Para evaluar los resultados, construya un gráfico correspondiente a la función de pérdida utilizada en el entrenamiento *versus* número de iteraciones (**realice 1000 iteraciones**), utilizando sólo el conjunto de entrenamiento (el objetivo de esta sección es familiarizarse con el algoritmo). Además de reportar el tiempo de entrenamiento mediante el algoritmo implementado en c) y d).

> e) Varié la tasa de aprendizaje $\eta \in [0,1]$ del algoritmo SGD del punto d), compare los resultados entre sí y con la solución óptima encontrada en c). Comente.

Para realizar la pregunta a, se importa la librería numpy. La función que se genera es lossFunction, la cual recibe como parámetro "y" que es un np.array que contiene los valores exactos de la función f(x) y "y_prima" es un np.array con la aproximación de la función f(x).

In [1]:
import numpy as np
def lossFunction(y, y_prima):
    resta = y - y_prima
    restaCuadrado = np.power(resta, 2)
    suma = np.sum(restaCuadrado)
    return suma / y.size

3.0

Para la pregunta b, se define la función LossFunctionGradient, la cual recive como parámetro el vector columna "y", eñ vectpr columna "b" que posee los coeficientes de la regresión lineal y "X" una matriz de NxP, donde N es la cantidad de inputs y P es la cantidad de parámetros del input más 1.

In [2]:
def LossFunctionGradient(y, b, X):
    # Arreglo donde se guardará el gradiente
    gradiente = np.array([0.0])
    # Número de ejemplos
    n = y.size
    # El primer for itera para obtener la derivada de cada variable B
    for i in range(b.size):
        # Variable donde se almacena el gradiente por partes de una variable de B
        gradParcial = 0
        # Este for es para recorrer todos los componentes de X y considerarlos en la derivada
        for c in range(n):
            gradParcial += (-2 / n) * (y[c] - np.dot(X[c], b))* X[c][i]
            
        if i == 0:
            gradiente[0] = gradParcial
        else:
            gradiente = np.append(gradiente, gradParcial)
    # Se retorna el gradiente de B
    return gradiente.reshape(-1, 1)

array([[2588.        ],
       [1252.66666667],
       [2952.        ],
       [ 384.        ]])

Para la pregunta c, se utiliza el resultado de la derivada igualada a 0: $(X^{T}X)^{-1}X^{T}Y = B$ para generar la función factoresB que recibe como parámetro el vector Y y la matriz X. Esto se logra a partir de las funciones de numpy que permiten obtener la transpuesta de una matriz, la inversa y poder multiplicar matrices. El retorno de esta función es igual a un vector columna que posee los factores de B.

In [3]:
def factoresB(Y, X):
    XT = X.transpose()
    # (X_T * X)^(-1)
    mult = np.dot(XT, X)
    inv = np.linalg.inv(mult)
    mult = np.dot(inv, XT)
    mult = np.dot(mult, Y)
    return mult

[[ 1.140625  ]
 [-3.14635768]
 [ 4.65625   ]
 [-9.84375   ]]


array([[ 3,  4,  5,  1],
       [ 6,  7,  8,  1],
       [ 9, 19, 11,  1]])

Para la pregunta d, se realiza una función que obtiene los coeficientes de B mediante SGD. Esta recibe como parámetro el vector columna y, la matriz X de dimensiones NxP, los valores iniciales del vector columna B, n que es la tasa de aprendizaje que por defecto es 0.001 y N que es la cantidad de iteraciones de la aproximación. Esta función se apoya en la función realizada anteriormente para calcular el gradiente de B (LossFunctionGradient).

In [4]:
def SGD(y, x, B, n = 0.001, N = 10):
    for i in range(N):
        B = B - n * LossFunctionGradient(y, B, x)
    return B

array([[21579.93890811],
       [39647.67162265],
       [27587.38189112],
       [ 3011.2214915 ]])

Para la pregunta e, se utiliza el dataset load_boston que contiene varios parámetros que determinan el precio de una propiedad. Primero se normalizan este dataset para tener una mejor estimación y que no se vea afectada por la diferencia de escalas. Luego, a los valores de entrenamiento X, se le agrega una columna de 1 al final para que se corresponda con la regresión lineal. Además, se genera un vector de coeficientes iniciales iguales a 0. Se ajusta un valor de tasa de entrenamiento igual a 0.0007. Luego, se utiliza la función SGD para obtener las estimaciones de los coeficientes de la regresión lineal. Esta se ocupa desde 1 a 1000 iteraciones y se guardan los resultados para así obtener los valores de y estimados. Con los valores de y_estimados e y reales, se utiliza la función lossFunction para cualcular el error cuadrático. Con esto se logra determinar el error para cada iteración y así graficarla. Además, se utiliza la librería Time para poder guardar el tiempo de computo de cada iteración.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import time



print(end - start)
X_train,y_train = load_boston(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# Se agrega columna de unos al final
X_train = np.c_[X_train, np.ones(506)]
# Vector inicial
B = np.array(([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]))

y_estimado = list()
tiempo = np.array([0.0])
# Primera iteración
start = time.time()
b = SGD(y_train, X_train, B, N = 1, n = 0.0007)
end = time.time()
tiempo[0] = end - start

y_estimado.append(np.dot(X_train, b))

for i in range(2, 1001):
    start = time.time()
    b = SGD(y_train, X_train, B, N = i, n = 0.0007)
    end = time.time()
    np.append(tiempo, (end - start))
    y_estimado.append(np.dot(X_train, b))
    
#n = 0.0007
#print(np.dot( X_train[10],SGD(y_train, X_train, B, N = 1000, n = 0.0007)))
#y_train
print(y_estimado)

0.04211115837097168


Para la pregunta f se genera un widget que permite cambiar la tasa de aprendizaje.

In [18]:
from ipywidgets import widgets, fixed
def widgetfunc(valor):
    Betas = SGD(y_train, X_train, B, N = 1000, n = valor)
    print(Betas)
    
sensibilidad = widgets.FloatLogSlider(
    value=10,
    base=10,
    min=-4, # max exponent of base
    max=-0.1, # min exponent of base
    step=0.1, # exponent step
    description='Tasa',
    continuous_update=False
    
)
widgets.interact(widgetfunc, valor = sensibilidad)

interactive(children=(FloatLogSlider(value=0.7943282347242815, continuous_update=False, description='Tasa', ma…

<function __main__.widgetfunc(valor)>

Como conclusión final de lo realizado, la convergencia del método depende mucho de la tasa de aprendizaje, es decir, que si se elige mal la tasa de aprendizaje no se podrá obtener una aproximación útil. En este caso, la tasa de aprendizaje converge cuando el valor de esta es pequeño (alrededor de 0.0007) pero si se hace muy pequeño, el método comienza de divergir y nuevamente no converge. También este método depende del valor inicial que se le de a B al comienzo del algoritmo. Si se elige uno inadecuado, puede que este no converja nunca nuevamente. Finalmente, este es un método útil cuando se sabe cual es la mejor tasa de entrenamiento y además de conocer aproximadamente que B es útil para que el método converja.

### Referencias
[1] Hastie, T.; Tibshirani, R., Friedman, J. (2009), *The Elements of Statistical Learning*, Second Edition.
Springer New York Inc.  
[2] http://statweb.stanford.edu/tibs/ElemStatLearn/datasets/  
[3] https://en.wikipedia.org/wiki/Stopwords  
[4] https://en.wikipedia.org/wiki/Stemming  
[5] https://en.wikipedia.org/wiki/Lemmatisation  
[6] http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature extraction.text  
[7] http://www.nltk.org/  
[8] https://gluon.mxnet.io/chapter06_optimization/gd-sgd-scratch.html  
[9] https://towardsdatascience.com/linear-regression-simplified-ordinary-least-square-vs-gradient-descent-48145de2cf76   
[10] https://en.wikipedia.org/wiki/Numerical_differentiation